<a href="https://colab.research.google.com/github/LinLin-LL/product_recommendation/blob/master/sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 44.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=784149c85b2db8950fc374f3a8009cb93ff2e6fb421e86e17c41d3f11832e501
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


## Import packages and load data

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
from google.colab import files
files.upload()

Saving Order.all.20200701_20200731.csv to Order.all.20200701_20200731.csv
Saving Order.all.20200801_20200831.csv to Order.all.20200801_20200831.csv
Saving Order.all.20200901_20200911.csv to Order.all.20200901_20200911.csv


{'Order.all.20200701_20200731.csv': b'Order ID,Order Status,Return / Refund Status,Tracking Number,Shipping Option,Shipment Method,Estimated Ship Out Date,Ship Time,Order Creation Date,Order Paid Time,Parent SKU Reference No.,Product Name,SKU Reference No.,Variation Name,Original Price,Deal Price,Quantity,Product Subtotal,Seller Rebate,Seller Discount,Shopee Rebate,SKU Total Weight,No of product in order,Order Total Weight,Voucher Code,Seller Voucher,Seller Absorbed Coin Cashback,Shopee Voucher,Bundle Deal Indicator,Shopee Bundle Discount,Seller Bundle Discount,Shopee Coins Offset,Credit Card Discount Total,Total Amount,Buyer Paid Shipping Fee,Transaction Fee,Commission Fee,Service Fee,Grand Total,Estimated Shipping Fee,Username (Buyer),Receiver Name,Phone Number,Delivery Address,Town,District,Area,State,Country,Zip Code,Remark from buyer,Order Complete Time,Note\r\n20072487RM0H9A,Completed,,MY203813597963E,Standard Delivery,Dropoff,2020-07-28 17:47,2020-07-26 05:34,2020-07-24 17:47,20

In [5]:
!ls

Order.all.20200701_20200731.csv  Order.all.20200901_20200911.csv
Order.all.20200801_20200831.csv  sample_data


In [6]:
spark = SparkSession \
    .builder \
    .appName("sales") \
    .config("spark.driver.maxResultSize", "96g") \
    .config("spark.driver.memory", "96g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

In [7]:
def read_data(csv_file):
  df = spark.read.csv(csv_file, header=True, inferSchema=True)
  return df

In [8]:
# data from July, August and September
sales_07 = read_data('Order.all.20200701_20200731.csv')
sales_08 = read_data('Order.all.20200801_20200831.csv')
sales_09 = read_data('Order.all.20200901_20200911.csv')

## Data cleaning

In [9]:
# only use the completed orders
def data_cleaning(df):
  completed_orders = df.filter(col('Order Status') == 'Completed')
  completed_orders = completed_orders.select('Username (Buyer)','Product Name','Quantity')
  return completed_orders

In [10]:
df_07 = data_cleaning(sales_07)
df_08 = data_cleaning(sales_08)
df_09 = data_cleaning(sales_09)

In [11]:
df = df_07.unionByName(df_08).unionByName(df_09)

In [12]:
df.printSchema()

root
 |-- Username (Buyer): string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Quantity: integer (nullable = true)



In [13]:
# transform user name to integer id
users = df.select('Username (Buyer)').distinct()
users = users.coalesce(1)
users = users.withColumn(
"userIntId", monotonically_increasing_id()).persist()
# users.show()

In [14]:
# transform product name to integer id
items = df.select('Product Name').distinct()
items = items.coalesce(1)
items = items.withColumn(
"itemIntId", monotonically_increasing_id()).persist()
# items.show()

In [15]:
sales_w_int_ids = df.join(
users, "Username (Buyer)", "left").join(items, "Product Name", "left")
# sales_w_int_ids.show()

In [16]:
sales_data = sales_w_int_ids.select(
                                        col("userIntId").alias("userId"),
                                        col("itemIntId").alias("itemId"),
                                        col("Quantity"))
# sales_data.show()

In [17]:
users = sales_data.select("userId").distinct()
items = sales_data.select("itemId").distinct()

cross_join = users.crossJoin(items).join(sales_data, ["userId", "itemId"], "left").fillna(0).persist()
# cross_join.show()

# ALS model

In [ ]:
from pyspark.sql.functions import rand
#from pyspark.ml.recommendation import ALS

In [22]:
#Expected percentile rank error metric function
def ROEM(predictions, userCol = userCol, itemCol = itemCol, ratingCol = ratingCol):
  #Creates table that can be queried
  predictions.createOrReplaceTempView("predictions")

  #Sum of total number of plays of all songs
  denominator = predictions.groupBy().sum(ratingCol).collect()[0][0]

  #Calculating rankings of songs predictions by user
  spark.sql("SELECT " + userCol + " , " + ratingCol + " , PERCENT_RANK() OVER (PARTITION BY " + userCol + " ORDER BY prediction DESC) AS rank FROM predictions").createOrReplaceTempView("rankings")

  #Multiplies the rank of each song by the number of plays and adds the products together
  numerator = spark.sql('SELECT SUM(' + ratingCol + ' * rank) FROM rankings').collect()[0][0]

  performance = numerator/denominator

  return performance

In [29]:
(train, test) = cross_join.randomSplit([.8, .2], seed=12)


In [45]:
# Empty list to be filled with models
model_list = []
params_list = []
roems = []

userCol = "userId"
itemCol = "itemId"
ratingCol = "Quantity"
ranks = [2,3,4,5]
maxIters = [5,10,15,20]
regParams = [0.05,.1]
alphas = [2,4]

In [46]:
# For loop will automatically create and store ALS models
for r in ranks:
  for mi in maxIters:
    for rp in regParams:
      for a in alphas:
        params_list.append({'rank': r, 'maxIter': mi, 'regParam': rp, 'alpha': a})
        model_list.append(ALS(userCol= userCol, itemCol= itemCol, ratingCol= ratingCol, 
                              rank = r, maxIter = mi, regParam = rp, alpha = a, 
                              coldStartStrategy="drop",nonnegative = True, implicitPrefs = True))

In [47]:
len(params_list)

64

In [ ]:
for model in model_list:
  # Fits each model to the training data
  trained_model = model.fit(train)
  # Generates test predictions
  predictions = trained_model.transform(test)
  # Evaluates each model's performance
  roems.append(ROEM(predictions))

In [43]:
roems

[0.015873015873015872, 0.015873015873015872, 0.0, 0.0, 0.0, 0.0]

In [44]:
params_list

[{'alpha': 2, 'maxIter': 5, 'rank': 3, 'regParam': 0.1},
 {'alpha': 2, 'maxIter': 10, 'rank': 3, 'regParam': 0.1},
 {'alpha': 2, 'maxIter': 5, 'rank': 4, 'regParam': 0.1},
 {'alpha': 2, 'maxIter': 10, 'rank': 4, 'regParam': 0.1},
 {'alpha': 2, 'maxIter': 5, 'rank': 5, 'regParam': 0.1},
 {'alpha': 2, 'maxIter': 10, 'rank': 5, 'regParam': 0.1}]

In [32]:
pr1 = tr.transform(test)

In [33]:
ROEM(pr1)

0.08730158730158728

In [35]:
tr2 = model_list[1].fit(train)
pr2 = tr2.transform(test)
roem2 = ROEM(pr2)
roem2

0.031746031746031744

In [36]:
tr3 = model_list[2].fit(train)
pr3 = tr3.transform(test)
roem3 = ROEM(pr3)
roem3

0.031746031746031744

In [49]:
m = ALS(userCol= userCol, itemCol= itemCol, ratingCol= ratingCol, 
                              rank = 6, maxIter = 25, regParam = 0.1, alpha = 3, 
                              coldStartStrategy="drop",nonnegative = True, implicitPrefs = True)
f = m.fit(train)
p = f.transform(test)
r = ROEM(p)
r

0.0

In [ ]:
0.015873015873015872#2,10

## Grid search + CV

In [ ]:
# Create training and test set (80/20 split)
(training, test) = cross_join.randomSplit([0.8, 0.2], seed = 12)

In [ ]:
# Build generic ALS model without hyperparameters
als = ALS(userCol=userCol, itemCol=itemCol, ratingCol=ratingCol,
coldStartStrategy="drop", nonnegative = True,
implicitPrefs = True)

In [ ]:
# Tell Spark what values to try for each hyperparameter
#from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().addGrid(als.rank, [2,3,4,5]).addGrid(als.maxIter, [5,10,15,20]).addGrid(als.regParam, [.1]).addGrid(als.alpha, [2]).build()